In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


## Notebook overview
- Get insight into tokenizer, tokens and doc lengths.
- Test different text truncation thresholds on the baseline.

#### Text truncation -- overview in tokenizer/doc lengths
- check how many docs exceed max

- first, tokenize text using tokenizer of mistral, geitje and Llama.
- Check if mistral and geitje indeed have the same tokenizer.
- After getting the tokens, check distribution and how many exceed max_threshold.

In [3]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")

In [9]:
from transformers import AutoTokenizer

def get_tokens(model_name, df, save_to_path, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_tokens = []
    all_tokens_len = []
    for txt in all_texts:
        tokens = tokenizer.tokenize(txt)
        all_tokens.append(tokens)
        all_tokens_len.append(len(tokens))

    df[new_col_name] = all_tokens
    df[f"count_{new_col_name}"] = all_tokens_len
    df.to_pickle(save_to_path)
    return df

# subdf = df.iloc[0:2]
# # display(subdf)
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', subdf, f"{cf.output_path}/try_out_token_count.pkl", 'text', 'token_count_geitje')

def fraction_token(df, max_token, token_len_col):
    for col in token_len_col:
        print(f"{len(df.loc[df[col]>max_token])} out of {len(df)} ({round(len(df.loc[df[col]>max_token])/len(df)*100, 2)}%) docs exceed a token length of {max_token}")

    for col in token_len_col:
        print(df[col].describe())

    


    

##### Get token lengths of model tokenizers

In [10]:
"""GEITje"""
df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
get_tokens('Rijgersberg/GEITje-7B-chat-v2', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'tokens_geitje')

"""Mistral"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'tokens_mistral')

"""Llama"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_token_length('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'tokens_llama2_7b_hf')

'Llama'

##### Analyse token length of model tokenizers

In [7]:
df_token_len = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
fraction_token(df_token_len, 4096, ['count_tokens_geitje', 'count_tokens_mistral', 'count_tokens_llama2_7b_hf'])

4005 out of 26704 (15.0%) docs exceed a token length of 4096
4005 out of 26704 (15.0%) docs exceed a token length of 4096
3839 out of 26704 (14.38%) docs exceed a token length of 4096
count     26704.000000
mean       4310.098749
std       14728.324592
min          75.000000
25%         680.000000
50%        1213.000000
75%        2609.000000
max      621995.000000
Name: token_count_geitje, dtype: float64
count     26704.000000
mean       4310.098749
std       14728.324592
min          75.000000
25%         680.000000
50%        1213.000000
75%        2609.000000
max      621995.000000
Name: token_count_mistral, dtype: float64
count     26704.000000
mean       4167.322124
std       14243.284787
min          74.000000
25%         662.000000
50%        1184.000000
75%        2530.000000
max      618067.000000
Name: token_count_llama2_7b_hf, dtype: float64


#### Test text truncation on baseline

In [11]:
tok = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
display(tok)

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,token_count_geitje,token_count_mistral,token_count_llama2_7b_hf,tokens_geitje,count_tokens_geitje
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,7049,val,> < Gemeente Raadsinformatiebrief\n| Amsterdam...,"[>, <, Gemeente, Raadsinformatiebrief, |, Amst...",2452,"[Gemeente, Raadsinformatiebrief, Amsterdam, Af...",1183,/home/azureuser/cloudfiles/code/blobfuse/raads...,5.0,Gemeente Raadsinformatiebrief Amsterdam Afdoen...,5085,5085,4924,"[▁>, ▁<, ▁Geme, ente, ▁Ra, ads, in, format, ie...",5085
1,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32682,train,Gemeente Amsterdam\n\n% Gemeenteraad R\nx% Gem...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, x, %...",1149,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",555,/home/azureuser/cloudfiles/code/blobfuse/raads...,3.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad S...,2351,2351,2259,"[▁Geme, ente, ▁Amsterdam, <0x0A>, <0x0A>, %, ▁...",2351
2,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15321,train,Agenda van de Raadscommissie Algemene Zaken\nv...,"[Agenda, van, de, Raadscommissie, Algemene, Za...",1215,"[Agenda, Raadscommissie, Algemene, Zaken, 20, ...",684,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0,Agenda Raadscommissie Algemene Zaken 20 juni 2...,3210,3210,3181,"[▁Ag, enda, ▁van, ▁de, ▁Ra, ads, comm, iss, ie...",3210
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2218,train,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",219,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",114,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,508,508,496,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",508
4,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33032,test,Verkeer & Meer VEN AES\nSamen veilig onderweg\...,"[Verkeer, &, Meer, VEN, AES, Samen, veilig, on...",1067,"[Verkeer, VEN, AES, Samen, veilig, onderweg, M...",575,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0,Verkeer VEN AES Samen veilig onderweg MEP KA R...,2259,2259,2235,"[▁Ver, ke, er, ▁&, ▁Me, er, ▁V, EN, ▁A, ES, <0...",2259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26699,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15352,train,X Gemeente Amsterdam\n% Stadsdeel West\nStadsd...,"[X, Gemeente, Amsterdam, %, Stadsdeel, West, S...",315,"[Gemeente, Amsterdam, Stadsdeel, West, Stadsde...",218,/home/azureuser/cloudfiles/code/blobfuse/raads...,4.0,Gemeente Amsterdam Stadsdeel West Stadsdeelcom...,1010,1010,1001,"[▁X, ▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Stad...",1010
26700,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4874,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",200,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",106,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,467,467,454,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",467
26701,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3803,train,x Gemeente Amsterdam R\n% Gemeenteraad\nGemeen...,"[x, Gemeente, Amsterdam, R, %, Gemeenteraad, G...",382,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",196,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,851,851,821,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, %, ▁...",851
26702,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,6178,train,2 Gemeente\nGemeenteraad RAAD\n% Amsterdam\nMo...,"[2, Gemeente, Gemeenteraad, RAAD, %, Amsterdam...",159,"[Gemeente, Gemeenteraad, RAAD, Amsterdam, Moti...",86,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Gemeenteraad RAAD Amsterdam Motie Dat...,376,376,372,"[▁, 2, ▁Geme, ente, <0x0A>, G, eme, ent, era, ...",376


In [17]:
def truncation(df,text_col, token_col_name, front_token_threshold, back_token_threshold=0):
    input = []
    for index, row in df.iterrows():
        # select text according to the token threshold -> first FRONT
        # select first n (= token_theshold) tokens using the model tokenizer
        tokens = row[token_col_name][0:front_token_threshold]

        # combine tokens into txt
        tokens_txt = ''.join(tokens)

        # \n is converted by tokenizer to <0x0A>, we reverse this to get original length
        len_char = len(tokens_txt.replace("<0x0A>", "\n")) # get character length

        # select the same amount of characters as the tokens
        front_txt = row[text_col][0:len_char]

        # Check if back of document also given as input
        if back_token_threshold != 0:
            # select LAST n (= token_theshold) tokens using the model tokenizer
            tokens = row[token_col_name][-back_token_threshold:]

            # combine tokens into txt
            tokens_txt = ''.join(tokens)

            # \n is converted by tokenizer to <0x0A>, we reverse this to get original length
            len_char = len(tokens_txt.replace("<0x0A>", "\n")) # get character length

            # select the same amount of characters as the tokens
            back_txt = row[text_col][-len_char:]

            # combine front and back text
            input_txt = front_txt + ' ' + back_txt

        else:
            input_txt = front_txt

        input.append(input_txt)

    df[f"TruncationFront{front_token_threshold}Back{back_token_threshold}"] = input
    return df




trunc = truncation(tok, 'text', 'tokens_geitje', 50,50)
display(trunc)

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,token_count_geitje,token_count_mistral,token_count_llama2_7b_hf,tokens_geitje,count_tokens_geitje,TruncationFront50Back50
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,7049,val,> < Gemeente Raadsinformatiebrief\n| Amsterdam...,"[>, <, Gemeente, Raadsinformatiebrief, |, Amst...",2452,"[Gemeente, Raadsinformatiebrief, Amsterdam, Af...",1183,/home/azureuser/cloudfiles/code/blobfuse/raads...,5.0,Gemeente Raadsinformatiebrief Amsterdam Afdoen...,5085,5085,4924,"[▁>, ▁<, ▁Geme, ente, ▁Ra, ads, in, format, ie...",5085,> < Gemeente Raadsinformatiebrief\n| Amsterdam...
1,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32682,train,Gemeente Amsterdam\n\n% Gemeenteraad R\nx% Gem...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, x, %...",1149,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",555,/home/azureuser/cloudfiles/code/blobfuse/raads...,3.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad S...,2351,2351,2259,"[▁Geme, ente, ▁Amsterdam, <0x0A>, <0x0A>, %, ▁...",2351,Gemeente Amsterdam\n\n% Gemeenteraad R\nx% Gem...
2,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15321,train,Agenda van de Raadscommissie Algemene Zaken\nv...,"[Agenda, van, de, Raadscommissie, Algemene, Za...",1215,"[Agenda, Raadscommissie, Algemene, Zaken, 20, ...",684,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0,Agenda Raadscommissie Algemene Zaken 20 juni 2...,3210,3210,3181,"[▁Ag, enda, ▁van, ▁de, ▁Ra, ads, comm, iss, ie...",3210,Agenda van de Raadscommissie Algemene Zaken\nv...
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2218,train,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",219,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",114,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,508,508,496,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",508,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...
4,Factsheets,/home/azureuser/cloudfiles/code/blobfuse/raads...,33032,test,Verkeer & Meer VEN AES\nSamen veilig onderweg\...,"[Verkeer, &, Meer, VEN, AES, Samen, veilig, on...",1067,"[Verkeer, VEN, AES, Samen, veilig, onderweg, M...",575,/home/azureuser/cloudfiles/code/blobfuse/raads...,6.0,Verkeer VEN AES Samen veilig onderweg MEP KA R...,2259,2259,2235,"[▁Ver, ke, er, ▁&, ▁Me, er, ▁V, EN, ▁A, ES, <0...",2259,Verkeer & Meer VEN AES\nSamen veilig onderweg\...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26699,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15352,train,X Gemeente Amsterdam\n% Stadsdeel West\nStadsd...,"[X, Gemeente, Amsterdam, %, Stadsdeel, West, S...",315,"[Gemeente, Amsterdam, Stadsdeel, West, Stadsde...",218,/home/azureuser/cloudfiles/code/blobfuse/raads...,4.0,Gemeente Amsterdam Stadsdeel West Stadsdeelcom...,1010,1010,1001,"[▁X, ▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Stad...",1010,X Gemeente Amsterdam\n% Stadsdeel West\nStadsd...
26700,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4874,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",200,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",106,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,467,467,454,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",467,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...
26701,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3803,train,x Gemeente Amsterdam R\n% Gemeenteraad\nGemeen...,"[x, Gemeente, Amsterdam, R, %, Gemeenteraad, G...",382,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",196,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,851,851,821,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, %, ▁...",851,x Gemeente Amsterdam R\n% Gemeenteraad\nGemeen...
26702,Motie,/h